##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 1.x vs TensorFlow 2 – Comportamentos e APIs

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/tf1_vs_tf2"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/tf1_vs_tf2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/tf1_vs_tf2.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/tf1_vs_tf2.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Nos bastidores, o TensorFlow 2 segue um paradigma de programação fundamentalmente diferente do TF1.x.

Este guia descreve as diferenças fundamentais entre TF1.x e TF2 em termos de comportamentos e APIs, e como tudo isso se relaciona com sua jornada de migração.

## Resumo de alto nível das principais mudanças

Fundamentalmente, TF1.x e TF2 apresentam um conjunto diferente de comportamentos em tempo de execução quanto ao modo de execução (eager no TF2), variáveis, fluxo de controle, formatos de tensor e comparações de igualdade entre tensores. Para ser compatível com TF2, seu código precisa ser compatível com o conjunto completo de comportamentos do TF2. Durante a migração, você poderá ativar ou desativar a maioria desses comportamentos individualmente através das APIs `tf.compat.v1.enable_*` ou `tf.compat.v1.disable_*`. A única exceção é a remoção de coleções, que é um efeito colateral da ativação/desativação da execução eager.

Em alto nível, o TensorFlow 2:

- Remove [APIs redundantes](https://github.com/tensorflow/community/blob/master/rfcs/20180827-api-names.md).
- Deixa as APIs mais consistentes - por exemplo, [RNNs unificadas](https://github.com/tensorflow/community/blob/master/rfcs/20180920-unify-rnn-interface.md) e [Otimizadores unificados](https://github.com/tensorflow/community/blob/master/rfcs/20181016-optimizer-unification.md).
- Prefere [funções em vez de sessões](https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20.md) e integra-se melhor ao runtime do Python com [execução eager](https://www.tensorflow.org/guide/eager) ativada por padrão junto com `tf.function` que fornece dependências de controle automático para grafos e compilação.
- Descontinua as [coleções](https://github.com/tensorflow/community/blob/master/rfcs/20180905-deprecate-collections.md) de grafos globais.
- Altera a semântica de concorrência de variáveis ​​usando [`ResourceVariables` em vez de `ReferenceVariables`](https://github.com/tensorflow/community/blob/master/rfcs/20180817-variables-20.md).
- Suporta [fluxo de controle](https://github.com/tensorflow/community/blob/master/rfcs/20180507-cond-v2.md) diferenciável e [baseado em funções](https://github.com/tensorflow/community/blob/master/rfcs/20180821-differentiable-functional-while.md) (fluxo de controle v2).
- Simplifica a API TensorShape para conter valores `int` em vez de objetos `tf.compat.v1.Dimension`.
- Atualiza a mecânica de igualdade de tensores. Em TF1.x, o operador `==` em tensores e variáveis ​​verificava a igualdade entre referências do objeto. No TF2 ele verifica a igualdade de valores. Além disso, não é maios possível fazer hash de tensores/variáveis, mas você pode fazer hash em referências de objetos para eles via `var.ref()` se precisar usá-los em conjuntos ou como chaves em um `dict`.

As seções abaixo fornecem mais contexto sobre as diferenças entre TF1.x e TF2. Para saber mais sobre o processo de design por trás do TF2, leia as [RFCs](https://github.com/tensorflow/community/pulls?utf8=%E2%9C%93&q=is%3Apr) e os [documentos de design](https://github.com/tensorflow/community/tree/master/rfcs).

## Limpeza de APIs

Muitas APIs ou [desapareceram ou foram movidas](https://github.com/tensorflow/community/blob/master/rfcs/20180827-api-names.md) no TF2. Algumas das principais mudanças incluem a remoção de `tf.app`, `tf.flags` e `tf.logging` em favor do agora do código aberto [absl-py](https://github.com/abseil/abseil-py), realojando projetos que viviam em `tf.contrib` e limpando o namespace principal `tf.*` ao mover funções menos usadas para subpacotes como `tf.math`. Algumas APIs foram substituídas por equivalentes no TF2 - `tf.summary`, `tf.keras.metrics` e `tf.keras.optimizers`.

### `tf.compat.v1`: endpoints de API legados e de compatibilidade

Os símbolos no contexto dos namespaces `tf.compat` e `tf.compat.v1` não são considerados APIs do TF2. Esses namespaces expõem uma mistura de símbolos de compatibilidade, bem como endpoints de API herdados do TF 1.x. O objetivo é ajudar na migração do TF1.x para o TF2. No entanto, como nenhuma dessas APIs `compat.v1` são APIs TF2 idiomáticas, elas não devem ser usadas para escrever código TF2 novo.

Símbolos `tf.compat.v1` individuais podem ser compatíveis com TF2 porque continuam a funcionar mesmo com comportamentos do TF2 habilitados (como `tf.compat.v1.losses.mean_squared_error`), enquanto outros são incompatíveis com TF2 (como `tf.compat.v1.metrics.accuracy`). Muitos símbolos `compat.v1` (embora não todos) contêm informações de migração dedicadas em sua documentação que explicam seu grau de compatibilidade com comportamentos do TF2, bem como como migrá-los para APIs do TF2.

O [script de upgrade do TF2](https://www.tensorflow.org/guide/migrate/upgrade) pode mapear muitos símbolos da API `compat.v1` para APIs do TF2 equivalentes no caso em que eles sejam aliases ou tenham os mesmos argumentos, mas com uma ordem diferente. Você também pode usar o script de upgrade para renomear automaticamente APIs TF1.x.

### Falsos cognatos

Há um conjunto de símbolos que são falsos cognatos encontrados no namespace `tf` do TF2 (não em `compat.v1`) e que na verdade ignoram os comportamentos do TF2 nos bastidores e/ou não são totalmente compatíveis com o conjunto completo de comportamentos do TF2. Como tal, é provável que essas APIs se comportem mal com o código TF2, potencialmente de forma silenciosa.

- `tf.estimator.*`: os estimadores criam e usam grafos e sessões nos bastidores. Como tal, estes não devem ser considerados compatíveis com TF2. Se o seu código executa estimadores, ele não está usando comportamentos do TF2.
- `keras.Model.model_to_estimator(...)`: isto cria um estimador nos bastidores, que, como mencionado acima, não é compatível com TF2.
- `tf.Graph().as_default()`: isto insere os comportamentos do grafo TF1.x e não segue os comportamentos `tf.function` padrão compatíveis com o TF2. O código que entra em grafos como este geralmente os executará via Sessions e não deve ser considerado compatível com TF2.
- `tf.feature_column.*` As APIs da coluna de características geralmente dependem da criação de variáveis `tf.compat.v1.get_variable` no estilo TF1 e assumem que as variáveis ​​criadas serão acessadas via coleções globais. Como o TF2 não oferece suporte a coleções, as APIs podem não funcionar corretamente ao executá-las com os comportamentos do TF2 habilitados.

### Outras alterações na API

- O TF2 traz melhorias significativas nos algoritmos de posicionamento de dispositivos, o que torna desnecessário o uso de `tf.colocate_with`. Se removê-lo causar degradação no desempenho, por favor, [registre um bug](https://github.com/tensorflow/tensorflow/issues).

- Substitua todo o uso de `tf.v1.ConfigProto` por funções equivalentes de `tf.config`.

## Execução antecipada (eager)

TF1.x exigia que os usuários construíssem manualmente uma [árvore de sintaxe abstrata](https://en.wikipedia.org/wiki/Abstract_syntax_tree) (o grafo), fazendo chamadas de API `tf.*` e, em seguida, compilassem manualmente a árvore de sintaxe abstrata, passando um conjunto de tensores de saída e tensores de entrada para uma chamada `session.run`. O TF2 é executado em modo eager (como o Python normalmente é) e isto faz com que grafos e as sessões pareçam detalhes de implementação.

Um subproduto notável da execution eager é que `tf.control_dependencies` não é mais necessário, já que todas as linhas de código são executadas sequencialmente (dentro de uma `tf.function`, os códigos com efeitos colaterais são executados na ordem em que foram escritos).

## O fim das variáveis globais

O TensorFlow 1.X dependia muito de namespaces implicitamente globais. Quando você chamava `tf.Variable`, ela era colocada no grafo padrão e permanecia lá, mesmo se você perdesse a variável do Python que apontava para ela. Depois, você poderia recuperar essa `tf.Variable`, mas somente se soubesse o nome com que foi criada originalmente. Isto era difícil de fazer se você não tivesse controle sobre a criação da variável. Como resultado, houve uma proliferação de mecanismos para tentar ajudar os usuários a encontrar suas variáveis novamente e para os frameworks encontrarem variáveis criadas por usuários: escopos de variáveis, coleções globais, métodos auxiliares, como `tf.get_global_step` e `tf.global_variables_initializer`, otimizadores computando gradientes implicitamente sobre todas as variáveis treináveis e assim por diante. O TensorFlow 2.0 elimina todos esses mecanismos ([Variables 2.0 RFC](https://github.com/tensorflow/community/pull/11)), dando lugar ao mecanismo padrão: manter o controle de suas variáveis! Se você perder o controle de uma `tf.Variable`, ela é recolhida pelo coletor de lixo.

O requisito de rastrear variáveis ​​cria algum trabalho extra, mas com ferramentas como os [shims de modelagem](./model_mapping.ipynb) e comportamentos como [coleções implícitas de variáveis ​​orientadas a objetos em `tf.Module` s e `tf.keras.layers.Layer`](https://www.tensorflow.org/guide/intro_to_modules), o trabalho é minimizado.

## Funções, não sessões

Uma chamada `session.run` é quase como uma chamada de função: você especifica as entradas e a função a ser chamada e recebe de volta um conjunto de saídas. No TF2, você pode decorar uma função Python usando `tf.function` para marcá-la para compilação JIT para que o TensorFlow a execute como um único grafo ([Functions 2.0 RFC](https://github.com/tensorflow/community/pull/20)). Este mecanismo permite que o TF2 obtenha todos os benefícios do modo grafo:

- Desempenho: A função pode ser otimizada (remoção de nós, fusão de kernel, etc.)
- Portabilidade: a função pode ser exportada/reimportada ([SavedModel 2.0 RFC](https://github.com/tensorflow/community/pull/34)), permitindo reutilizar e compartilhar funções modulares do TensorFlow.

```python
# TF1.x
outputs = session.run(f(placeholder), feed_dict={placeholder: input})
# TF2
outputs = f(input)
```

Com o poder de intercalar livremente o código Python e TensorFlow, você pode aproveitar a expressividade do Python. No entanto, o TensorFlow portátil é executado em contextos sem um interpretador Python, como dispositivos móveis, C++ e JavaScript. Para ajudar a evitar reescrever seu código ao adicionar `tf.function`, use [AutoGraph](https://tensorflow.org/guide/function) para converter um subconjunto de construtos do Python em seus equivalentes do TensorFlow:

- `for`/`while` -&gt; `tf.while_loop` (`break` e `continue` são suportados)
- `if` -&gt; `tf.cond`
- `for _ in dataset` -&gt; `dataset.reduce`

O AutoGraph oferece suporte a aninhamentos arbitrários de fluxo de controle, o que torna possível implementar de maneira concisa e com bom desempenho muitos programas complexos de aprendizado de máquina, como modelos de sequência, aprendizado por reforço, loops de treinamento personalizados e muito mais.

## Adaptação mudanças de comportamento do TF 2.x

Sua migração para o TF2 só estará concluída depois que você migrar para o conjunto completo de comportamentos do TF2. O conjunto completo de comportamentos pode ser ativado ou desativado via `tf.compat.v1.enable_v2_behaviors` e `tf.compat.v1.disable_v2_behaviors`. As seções abaixo discutem detalhadamente cada mudança importante de comportamento.

### Usando `tf.function`

As maiores mudanças em seus programas durante a migração provavelmente virão da mudança fundamental de paradigma do modelo de programação de grafos e sessões para execução eager e `tf.function`. Consulte os [guias de migração do TF2](https://tensorflow.org/guide/migrate) para saber mais sobre como migrar de APIs incompatíveis com execução eager e `tf.function` para APIs compatíveis.

Observação: durante a migração, você poderá optar por ativar e desativar diretamente a execução eager com `tf.compat.v1.enable_eager_execution` e `tf.compat.v1.disable_eager_execution`, mas isto só pode ser feito uma vez durante a vida útil do seu programa.

Abaixo estão alguns padrões de programação comuns não vinculados a nenhuma API que podem causar problemas ao trocar objetos `tf.Graph` por `tf.compat.v1.Session` para execução eager com objetos `tf.function`.

#### Padrão 1: manipulação de objetos Python e variáveis ​​destinadas a serem criadas apenas uma vez e executadas múltiplas vezes

<a id="pattern-1"></a>

Em programas TF1.x que dependem de grafos e sessões, a expectativa geralmente é que toda a lógica Python no seu programa seja executada apenas uma vez. No entanto, com execução eager e `tf.function` é justo esperar que sua lógica Python seja executada pelo menos uma vez, mas possivelmente mais vezes (várias vezes em modo eager ou várias vezes em diferentes rastreamentos de `tf.function`). Às vezes, `tf.function` irá rastrear duas vezes na mesma entrada, causando comportamentos inesperados (veja os Exemplos 1 e 2). Consulte o <a>guia</a> <code>tf.function</code> para obter mais detalhes.

Observação: Esse padrão geralmente faz com que seu código se comporte mal silenciosamente ao executar de forma eager sem objetos `tf.function`, mas geralmente gera um `InaccessibleTensorError` ou um `ValueError` ao tentar empacotar o código problemático dentro de um `tf.function`. Para descobrir e depurar esse problema, é recomendável empacotar seu código com `tf.function` desde o início e usar [pdb](https://docs.python.org/3/library/pdb.html) ou depuração interativa para identificar a origem do `InaccessibleTensorError`.

**Exemplo 1: criação de variáveis**

Considere o exemplo abaixo, onde a função cria uma variável quando chamada:

```python
def f():
  v = tf.Variable(1.0)
  return v

with tf.Graph().as_default():
  with tf.compat.v1.Session() as sess:
    res = f()
    sess.run(tf.compat.v1.global_variables_initializer())
    sess.run(res)
```

Não é permitido, porém, empacotar ingenuamente a função acima que contém a criação de variáveis ​​com `tf.function`, pois `tf.function` suporta apenas [criações de variáveis ​​​​singleton na primeira chamada](https://www.tensorflow.org/guide/function#creating_tfvariables). Para garantir isso, quando tf.function detectar a criação de variável na primeira chamada, ele tentará rastrear novamente e gerará um erro se acontecer a criação da variável no segundo rastreamento.

```python
@tf.function
def f():
  print("trace") # This will print twice because the python body is run twice
  v = tf.Variable(1.0)
  return v

try:
  f()
except ValueError as e:
  print(e)
```

Uma solução alternativa é armazenar em cache e reutilizar a variável depois de ela ser criada na primeira chamada.

```python
class Model(tf.Module):
  def __init__(self):
    self.v = None

  @tf.function
  def __call__(self):
    print("trace") # This will print twice because the python body is run twice
    if self.v is None:
      self.v = tf.Variable(0)
    return self.v

m = Model()
m()
```

**Exemplo 2: tensores fora do escopo devido ao rastreamento repetido de `tf.function`**

Conforme demonstrado no Exemplo 1, `tf.function` irá rastrear novamente quando detectar a criação de variável na primeira chamada. Isto pode causar uma confusão extra, pois os dois rastreamentos criarão dois grafos. Quando o grafo criado no segundo rastreamento tenta acessar um Tensor do grafo gerado durante o primeiro rastreamento, o Tensorflow lançará um erro reclamando que o Tensor está fora do escopo. Para demonstrar o cenário, o código abaixo cria um dataset na primeira chamada `tf.function`. Isto iria executar como esperado.

```python
class Model(tf.Module):
  def __init__(self):
    self.dataset = None

  @tf.function
  def __call__(self):
    print("trace") # This will print once: only traced once
    if self.dataset is None:
      self.dataset = tf.data.Dataset.from_tensors([1, 2, 3])
    it = iter(self.dataset)
    return next(it)

m = Model()
m()
```

No entanto, se também tentarmos criar uma variável na primeira chamada `tf.function`, o código produzirá um erro reclamando que o dataset está fora do escopo. Isto ocorre porque o dataset está no primeiro grafo, enquanto que o segundo grafo também tenta acessá-lo.

```python
class Model(tf.Module):
  def __init__(self):
    self.v = None
    self.dataset = None

  @tf.function
  def __call__(self):
    print("trace") # This will print twice because the python body is run twice
    if self.v is None:
      self.v = tf.Variable(0)
    if self.dataset is None:
      self.dataset = tf.data.Dataset.from_tensors([1, 2, 3])
    it = iter(self.dataset)
    return [self.v, next(it)]

m = Model()
try:
  m()
except TypeError as e:
  print(e) # <tf.Tensor ...> is out of scope and cannot be used here.
```

A solução mais simples é garantir que a criação da variável e a criação do dataset estejam fora da chamada `tf.function`. Por exemplo:

```python
class Model(tf.Module):
  def __init__(self):
    self.v = None
    self.dataset = None

  def initialize(self):
    if self.dataset is None:
      self.dataset = tf.data.Dataset.from_tensors([1, 2, 3])
    if self.v is None:
      self.v = tf.Variable(0)

  @tf.function
  def __call__(self):
    it = iter(self.dataset)
    return [self.v, next(it)]

m = Model()
m.initialize()
m()
```

No entanto, às vezes não há como evitar a criação de variáveis ​​em `tf.function` (como variáveis ​​de slot em alguns [otimizadores TF keras](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Optimizer#slots)). Ainda assim, podemos simplesmente mover a criação do dataset para fora da chamada `tf.function`. A razão pela qual podemos confiar nisso é porque `tf.function` receberá o dataset como uma entrada implícita e ambos os grafos poderão acessá-lo corretamente.

```python
class Model(tf.Module):
  def __init__(self):
    self.v = None
    self.dataset = None

  def initialize(self):
    if self.dataset is None:
      self.dataset = tf.data.Dataset.from_tensors([1, 2, 3])

  @tf.function
  def __call__(self):
    if self.v is None:
      self.v = tf.Variable(0)
    it = iter(self.dataset)
    return [self.v, next(it)]

m = Model()
m.initialize()
m()
```

**Exemplo 3: recriações inesperadas de objetos Tensorflow devido ao uso de dict**

`tf.function` tem suporte muito ruim para efeitos colaterais do python, como anexar a uma lista ou verificar/adicionar a um dicionário. Mais detalhes estão em ["Melhor desempenho com tf.function"](https://www.tensorflow.org/guide/function#executing_python_side_effects). No exemplo abaixo, o código usa dicionários para armazenar datasets e iteradores em cache. Para a mesma chave, cada chamada ao modelo retornará o mesmo iterador do dataset.

```python
class Model(tf.Module):
  def __init__(self):
    self.datasets = {}
    self.iterators = {}

  def __call__(self, key):
    if key not in self.datasets:
      self.datasets[key] = tf.compat.v1.data.Dataset.from_tensor_slices([1, 2, 3])
      self.iterators[key] = self.datasets[key].make_initializable_iterator()
    return self.iterators[key]

with tf.Graph().as_default():
  with tf.compat.v1.Session() as sess:
    m = Model()
    it = m('a')
    sess.run(it.initializer)
    for _ in range(3):
      print(sess.run(it.get_next())) # prints 1, 2, 3
```

No entanto, o padrão acima não funcionará como esperado em `tf.function`. Durante o rastreamento, `tf.function` irá ignorar o efeito colateral do Python da adição aos dicionários. Em vez disso, ele apenas lembra a criação de um novo dataset e iterador. Como resultado, cada chamada ao modelo sempre retornará um novo iterador. Este problema é difícil de perceber, a menos que os resultados numéricos ou o desempenho sejam suficientemente significativos. Portanto, recomendamos que os usuários pensem cuidadosamente no código antes de empacotar `tf.function` ingenuamente em código python.

```python
class Model(tf.Module):
  def __init__(self):
    self.datasets = {}
    self.iterators = {}

  @tf.function
  def __call__(self, key):
    if key not in self.datasets:
      self.datasets[key] = tf.data.Dataset.from_tensor_slices([1, 2, 3])
      self.iterators[key] = iter(self.datasets[key])
    return self.iterators[key]

m = Model()
for _ in range(3):
  print(next(m('a'))) # prints 1, 1, 1
```

Podemos usar [`tf.init_scope`](https://www.tensorflow.org/api_docs/python/tf/init_scope) para elevar o dataset e a criação do iterador para fora do grafo, e assim alcançar o comportamento esperado:

```python
class Model(tf.Module):
  def __init__(self):
    self.datasets = {}
    self.iterators = {}

  @tf.function
  def __call__(self, key):
    if key not in self.datasets:
      # Lifts ops out of function-building graphs
      with tf.init_scope():
        self.datasets[key] = tf.data.Dataset.from_tensor_slices([1, 2, 3])
        self.iterators[key] = iter(self.datasets[key])
    return self.iterators[key]

m = Model()
for _ in range(3):
  print(next(m('a'))) # prints 1, 2, 3
```

A regra geral é evitar depender dos efeitos colaterais do Python em sua lógica e usá-los apenas para depurar seus rastreamentos.

**Exemplo 4: manipulando uma lista Python global**

O código TF1.x a seguir usa uma lista global de perdas que é usada apenas para manter a lista de perdas geradas pelo passo de treinamento atual. Observe que a lógica Python que anexa perdas à lista será chamada apenas uma vez, independentemente de quantos passos de treinamento forem executados para a sessão.

```python
all_losses = []

class Model():
  def __call__(...):
    ...
    all_losses.append(regularization_loss)
    all_losses.append(label_loss_a)
    all_losses.append(label_loss_b)
    ...

g = tf.Graph()
with g.as_default():
  ...
  # initialize all objects
  model = Model()
  optimizer = ...
  ...
  # train step
  model(...)
  total_loss = tf.reduce_sum(all_losses)
  optimizer.minimize(total_loss)
  ...
...
sess = tf.compat.v1.Session(graph=g)
sess.run(...)
```

No entanto, se esta lógica Python for ingenuamente mapeada para TF2 com execução eager, a lista global de perdas terá novos valores anexados a ela em cada passo de treinamento. Isto significa que o código do passo de treinamento, que anteriormente esperava que a lista contivesse apenas as perdas do passo de treinamento atual, agora vê na verdade a lista de perdas de todos os passos de treinamento executados até o momento. Esta é uma mudança de comportamento não intencional, e a lista precisará ser apagada no início de cada passo ou tornada local no passo de treinamento.

```python
all_losses = []

class Model():
  def __call__(...):
    ...
    all_losses.append(regularization_loss)
    all_losses.append(label_loss_a)
    all_losses.append(label_loss_b)
    ...

# initialize all objects
model = Model()
optimizer = ...

def train_step(...)
  ...
  model(...)
  total_loss = tf.reduce_sum(all_losses) # global list is never cleared,
  # Accidentally accumulates sum loss across all training steps
  optimizer.minimize(total_loss)
  ...
```

#### Padrão 2: um tensor simbólico destinado a ser recomputado a cada passo no TF1.x é acidentalmente armazenado em cache com o valor inicial ao mudar para o modo eager.

<a id="pattern-2"></a>

Este padrão geralmente faz com que seu código se comporte mal silenciosamente ao executar de forma eager fora de objetos tf.function, mas gera um `InaccessibleTensorError` se o cache do valor inicial ocorrer dentro de um `tf.function`. No entanto, esteja ciente de que, para evitar o [Padrão 1](#pattern-1) acima, você muitas vezes estruturará inadvertidamente seu código de forma que esse cache de valor inicial aconteça *fora* de qualquer `tf.function` que possa gerar um erro. Portanto, tome cuidado adicional se souber que seu programa pode ser suscetível a esse padrão.

A solução geral para esse padrão é reestruturar o código ou usar callables do Python, se necessário, para garantir que o valor seja recalculado a cada vez, em vez de ser armazenado em cache acidentalmente.

**Exemplo 1: Taxa de aprendizagem/hiperparâmetro/etc. cronogramas que dependem de um passo global**

No trecho de código a seguir, a expectativa é que toda vez que a sessão for executada, o valor `global_step` mais recente seja lido e uma nova taxa de aprendizado seja computada.

```python
g = tf.Graph()
with g.as_default():
  ...
  global_step = tf.Variable(0)
  learning_rate = 1.0 / global_step
  opt = tf.compat.v1.train.GradientDescentOptimizer(learning_rate)
  ...
  global_step.assign_add(1)
...
sess = tf.compat.v1.Session(graph=g)
sess.run(...)
```

No entanto, ao tentar mudar para eager, tenha cuidado para não acabar com a taxa de aprendizagem sendo computada apenas uma vez e depois reutilizada, em vez de seguir o cronograma pretendido:

```python
global_step = tf.Variable(0)
learning_rate = 1.0 / global_step # Wrong! Only computed once!
opt = tf.keras.optimizers.SGD(learning_rate)

def train_step(...):
  ...
  opt.apply_gradients(...)
  global_step.assign_add(1)
  ...
```

Como este exemplo específico é um padrão comum e os otimizadores devem ser inicializados apenas uma vez, e não em cada passo de treinamento, os otimizadores TF2 suportam cronogramas `tf.keras.optimizers.schedules.LearningRateSchedule` ou chamadas Python como argumentos para a taxa de aprendizagem e outros hiperparâmetros.

**Exemplo 2: inicializações simbólicas de números aleatórios passados como atributos de objetos e reutilizados via ponteiros são acidentalmente armazenados em cache ao mudar para o modo eager**

Considere o seguinte módulo `NoiseAdder`:

```python
class NoiseAdder(tf.Module):
  def __init__(shape, mean):
    self.noise_distribution = tf.random.normal(shape=shape, mean=mean)
    self.trainable_scale = tf.Variable(1.0, trainable=True)
  
  def add_noise(input):
    return (self.noise_distribution + input) * self.trainable_scale
```

Usá-lo da seguinte maneira em TF1.x calculará um novo tensor de ruído aleatório toda vez que a sessão for executada:

```python
g = tf.Graph()
with g.as_default():
  ...
  # initialize all variable-containing objects
  noise_adder = NoiseAdder(shape, mean)
  ...
  # computation pass
  x_with_noise = noise_adder.add_noise(x)
  ...
...
sess = tf.compat.v1.Session(graph=g)
sess.run(...)
```

No entanto, no TF2, inicializar o `noise_adder` no início fará com que o `noise_distribution` seja computado apenas uma vez e fique congelado para todos os passos de treinamento:

```python
...
# initialize all variable-containing objects
noise_adder = NoiseAdder(shape, mean) # Freezes `self.noise_distribution`!
...
# computation pass
x_with_noise = noise_adder.add_noise(x)
...
```

Para corrigir isso, refatore `NoiseAdder` para chamar `tf.random.normal` toda vez que um novo tensor aleatório for necessário, em vez de se referir ao mesmo objeto tensor todas as vezes.

```python
class NoiseAdder(tf.Module):
  def __init__(shape, mean):
    self.noise_distribution = lambda: tf.random.normal(shape=shape, mean=mean)
    self.trainable_scale = tf.Variable(1.0, trainable=True)
  
  def add_noise(input):
    return (self.noise_distribution() + input) * self.trainable_scale
```

#### Padrão 3: o código TF1.x depende diretamente de tensores e os procura por nome

<a id="pattern-3"></a>

É comum que os testes de código TF1.x dependam da verificação de quais tensores ou operações estão presentes em um grafo. Em alguns casos raros, o código de modelagem também dependerá dessas pesquisas por nome.

Os nomes dos tensores não são gerados durante a execução eager fora de `tf.function`, portanto, todos os usos de `tf.Tensor.name` devem acontecer dentro de um `tf.function`. Tenha em mente que os nomes reais gerados provavelmente serão diferentes entre TF1.x e TF2, mesmo dentro do mesmo `tf.function`, e as garantias da API não garantem a estabilidade dos nomes gerados em diferentes versões do TF.

Nota: nomes de variáveis ​​ainda são gerados mesmo fora de `tf.function`, mas também não é garantido que seus nomes correspondam entre TF1.x e TF2, exceto ao seguir as seções relevantes do [guia de mapeamento de modelos](./model_mapping.ipynb).


#### Padrão 4: a sessão TF1.x executa seletivamente apenas parte do grafo gerado

<a id="pattern-4"></a>

No TF1.x, você pode construir um grafo e, em seguida, optar por executar seletivamente apenas um subconjunto dele com uma sessão, escolhendo um conjunto de entradas e saídas que não exigem a execução de cada operação no grafo.

Por exemplo, você pode ter um gerador e um discriminador dentro de um único grafo e usar chamadas `tf.compat.v1.Session.run` separadas para alternar entre treinar apenas o discriminador ou apenas treinar o gerador.

No TF2, devido às dependências de controle automático em `tf.function` e à execução eager, não há pruning seletivo dos rastreamentos `tf.function`. Um grafo completo contendo todas as atualizações de variáveis ​​seria executado mesmo se, por exemplo, apenas a saída do discriminador ou do gerador fosse gerada como saída do `tf.function`.

Portanto, você precisaria ou usar múltiplos `tf.function` contendo diferentes partes do programa ou um argumento condicional para o `tf.function` que você poderia usar para decidir executar apenas as coisas que você realmente quer que sejam executadas.

### Remoção de coleções

Quando a execução eager está ativada, as APIs `compat.v1` relacionadas à coleção de grafos (incluindo aquelas que leem ou gravam em coleções ocultas, como `tf.compat.v1.trainable_variables`) não estarão mais disponíveis. Algumas podem gerar `ValueError`, enquanto outras podem retornar silenciosamente listas vazias.

O uso mais comum de coleções no TF1.x é manter inicializadores, o passo global, pesos, perdas de regularização, perdas de saída do modelo e atualizações de variáveis ​​que precisam ser executadas, como nas camadas `BatchNormalization`.

Veja como lidar com cada um desses usos comuns:

1. Inicializadores - ignore. A inicialização manual de variáveis ​​não é necessária com a execução eager ativada.
2. Passo global – consulte a documentação de `tf.compat.v1.train.get_or_create_global_step` para obter instruções de migração.
3. Pesos - mapeie seus modelos para objetos `tf.Module`/ `tf.keras.layers.Layer`/ `tf.keras.Model` seguindo as orientações no [guia de mapeamento de modelos](./model_mapping.ipynb) e, em seguida, use seus respectivos mecanismos de rastreamento de pesos, como `tf.module.trainable_variables`.
4. Perdas de regularização – mapeie seus modelos para objetos `tf.Module`/ `tf.keras.layers.Layer`/ `tf.keras.Model` seguindo as orientações no [guia de mapeamento de modelos](./model_mapping.ipynb) e, em seguida, use `tf.keras.losses`. Alternativamente, você também pode rastrear manualmente suas perdas de regularização.
5. Perdas de saída do modelo - use mecanismos de gerenciamento de perdas `tf.keras.Model` ou rastreie separadamente suas perdas sem usar coleções.
6. Atualizações de peso – ignore esta coleção. Execução eager e `tf.function` (com dependências de autógrafo e controle automático) significam que todas as atualizações de variáveis ​​serão executadas automaticamente. Portanto, você não terá que executar explicitamente todas as atualizações de peso no final, mas observe que isto significa que as atualizações de peso podem acontecer num momento diferente do que aconteciam no seu código TF1.x, dependendo de como você estava usando as dependências de controle.
7. Resumos – Veja o [guia da API sobre migração de resumos](https://www.tensorflow.org/tensorboard/migrate).

O uso de coleções mais complexas (como o uso de coleções personalizadas) pode exigir que você refatore seu código para manter seus próprios armazenamentos globais ou para que ele não dependa de nenhum armazenamento global.

### `ResourceVariables` em vez de `ReferenceVariables`

`ResourceVariables` têm garantias de consistência de leitura e gravação mais fortes do que `ReferenceVariables`. Isso leva a uma semântica mais previsível e mais fácil de entender se você observará ou não o resultado de uma gravação anterior ao usar suas variáveis. É extremamente improvável que essa alteração faça com que o código existente gere erros ou falhe silenciosamente.

No entanto, é ***possível, embora improvável,*** que essas garantias de consistência mais fortes aumentem o uso de memória do seu programa específico. Registre um [issue](https://github.com/tensorflow/tensorflow/issues) se você achar que esse é o caso. Além disso, se você tiver testes de unidade que dependam de comparações exatas de strings com os nomes dos operadores em um grafo correspondente às leituras de variáveis, esteja ciente de que a ativação de variáveis ​​de recursos poderá alterar ligeiramente os nomes desses operadores.

Para isolar o impacto dessa mudança de comportamento em seu código, se a execução eager estiver ativada, você pode usar `tf.compat.v1.disable_resource_variables()` e `tf.compat.v1.enable_resource_variables()` para desativar ou ativar globalmente essa mudança de comportamento. As `ResourceVariables` sempre serão usadas ​​se a execução eager estiver ativada.


### Controle de fluxo v2

No TF1.x, as ops de controle de fluxo são ops como `tf.cond` e `tf.while_loop`, operações de baixo nível inline, como `Switch`, `Merge`, etc. O TF2 fornece operações de controle de fluxo com funcionalidade aprimorada que são implementadas com rastreamentos `tf.function` separados para cada ramo e que suportam diferenciação de ordem superior.

Para isolar o impacto dessa mudança de comportamento em seu código, se a execução eager estiver desativada, você pode usar `tf.compat.v1.disable_control_flow_v2()` e `tf.compat.v1.enable_control_flow_v2()` para desativar ou ativar globalmente essa mudança de comportamento. No entanto, você só poderá desativar o fluxo de controle v2 se a execução eager também estiver desativada. Se estiver ativada, o fluxo de controle v2 sempre será utilizado.

Esta mudança de comportamento pode alterar drasticamente a estrutura dos programas TF gerados que usam fluxo de controle, pois eles conterão vários rastreamento de função aninhados em vez de um grafo plano. Portanto, qualquer código que seja altamente dependente da semântica exata dos rastreamentos produzidos pode exigir algumas modificações, que incluem:

- Código que depende de nomes de operadores e tensores
- Código que possui referências a tensores criados dentro de um ramo de fluxo de controle do TensorFlow chamadas de fora desse ramo. É provável que isto produza um `InaccessibleTensorError`

É esperado que essa mudança de comportamento tenha uma mudança de desempenho neutro ou positivo, mas se você encontrar um problema em que o fluxo de controle v2 tem um desempenho pior do que o fluxo de controle TF1.x, registre um [issue](https://github.com/tensorflow/tensorflow/issues) descrevendo as etapas de reprodução do problema. 

## Mudanças de comportamento na API TensorShape

A classe `TensorShape` foi simplificada para conter valores `int`, em vez de objetos `tf.compat.v1.Dimension`. Portanto, não há necessidade de chamar `.value` para obter um `int`.

Objetos `tf.compat.v1.Dimension` individuais ainda podem ser acessados ​​em `tf.TensorShape.dims`.

Para isolar o impacto dessa mudança de comportamento em seu código, você pode usar `tf.compat.v1.disable_v2_tensorshape()` e `tf.compat.v1.enable_v2_tensorshape()` para desativar ou ativar globalmente essa mudança de comportamento.

A seguir demonstramos as diferenças entre TF1.x e TF2.

In [ ]:
import tensorflow as tf

In [ ]:
# Create a shape and choose an index
i = 0
shape = tf.TensorShape([16, None, 256])
shape

Se você tinha isto no TF1.x:

```python
value = shape[i].value
```

Então faça isto no TF2:


In [ ]:
value = shape[i]
value

Se você tinha isto no TF1.x:

```python
for dim in shape:
    value = dim.value
    print(value)
```

Então faça isto no TF2:

In [ ]:
for value in shape:
  print(value)

Se você tinha isto no TF1.x (ou usou qualquer outro método de dimensão):

```python
dim = shape[i]
dim.assert_is_compatible_with(other_dim)
```

Então faça isto no TF2:

In [ ]:
other_dim = 16
Dimension = tf.compat.v1.Dimension

if shape.rank is None:
  dim = Dimension(None)
else:
  dim = shape.dims[i]
dim.is_compatible_with(other_dim) # or any other dimension method

In [ ]:
shape = tf.TensorShape(None)

if shape:
  dim = shape.dims[i]
  dim.is_compatible_with(other_dim) # or any other dimension method

O valor booleano de um `tf.TensorShape` é `True` se a classificação for conhecida, `False` caso contrário.

In [ ]:
print(bool(tf.TensorShape([])))      # Scalar
print(bool(tf.TensorShape([0])))     # 0-length vector
print(bool(tf.TensorShape([1])))     # 1-length vector
print(bool(tf.TensorShape([None])))  # Unknown-length vector
print(bool(tf.TensorShape([1, 10, 100])))       # 3D tensor
print(bool(tf.TensorShape([None, None, None]))) # 3D tensor with no known dimensions
print()
print(bool(tf.TensorShape(None)))  # A tensor with unknown rank.

### Erros potenciais devido a alterações no TensorShape

É improvável que as mudanças de comportamento do TensorShape quebrem silenciosamente seu código. No entanto, você poderá perceber que códigos relacionados a formatos começam a gerar `AttributeError`, pois valores`int` e `None` não têm os mesmos atributos que objetos `tf.compat.v1.Dimension`. Abaixo estão alguns exemplos de `AttributeError`:

In [ ]:
try:
  # Create a shape and choose an index
  shape = tf.TensorShape([16, None, 256])
  value = shape[0].value
except AttributeError as e:
  # 'int' object has no attribute 'value'
  print(e)

In [ ]:
try:
  # Create a shape and choose an index
  shape = tf.TensorShape([16, None, 256])
  dim = shape[1]
  other_dim = shape[2]
  dim.assert_is_compatible_with(other_dim)
except AttributeError as e:
  # 'NoneType' object has no attribute 'assert_is_compatible_with'
  print(e)

## Igualdade de tensores por valor

Os operadores binários `==` e `!=` em variáveis ​​e tensores foram alterados para comparar por valor no TF2 em vez de comparar por referência de objeto como em TF1.x. Além disso, não é mais possível fazer hash em tensores e variáveis ​​​​ou utilizá-los ​​em conjuntos ou chaves de dicts, porque pode não ser possível fazer hash deles por valor. Em vez disso, eles expõem um método `.ref()` que você pode usar para obter uma referência com hash ao tensor ou variável.

Para isolar o impacto dessa mudança de comportamento, você pode usar `tf.compat.v1.disable_tensor_equality()` e `tf.compat.v1.enable_tensor_equality()` para desativar ou ativar globalmente essa mudança de comportamento.

Por exemplo, no TF1.x, duas variáveis ​​com o mesmo valor retornarão falso quando você usar o operador `==`:

In [ ]:
tf.compat.v1.disable_tensor_equality()
x = tf.Variable(0.0)
y = tf.Variable(0.0)

x == y

Enquanto que no TF2 com verificações de igualdade de tensor ativadas, `x == y` retornará `True`.

In [ ]:
tf.compat.v1.enable_tensor_equality()
x = tf.Variable(0.0)
y = tf.Variable(0.0)

x == y

Então, no TF2, se você precisar comparar por referência de objeto, certifique-se de usar `is` e `is not`

In [ ]:
tf.compat.v1.enable_tensor_equality()
x = tf.Variable(0.0)
y = tf.Variable(0.0)

x is y

### Hash em tensores e variáveis

Com os comportamentos do TF1.x, você costumava adicionar variáveis ​​​​e tensores diretamente a estruturas de dados que exigem hashing, como chaves para objetos `set` e `dict`.

```python
x = tf.Variable(0.0)
set([x, tf.constant(2.0)])
```

No entanto, no TF2 com igualdade de tensores ativada, não é possível fazer hash de tensores e variáveis ​​devido à mudança da semântica do operador `==` e `!=` para verificações de igualdade de valor.

In [ ]:
tf.compat.v1.enable_tensor_equality()
x = tf.Variable(0.0)

try:
  set([x, tf.constant(2.0)])
except TypeError as e:
  # TypeError: Variable is unhashable. Instead, use tensor.ref() as the key.
  print(e)

Portanto, no TF2, se você precisar usar tensores ou objetos variáveis ​​​​como chaves ou conteúdo de um `set`, poderá usar `tensor.ref()` para obter uma referência com hash que poderá ser usada como chave:

In [ ]:
tf.compat.v1.enable_tensor_equality()
x = tf.Variable(0.0)

tensor_set = set([x.ref(), tf.constant(2.0).ref()])
assert x.ref() in tensor_set

tensor_set

Se necessário, você também pode obter o tensor ou variável da referência usando `reference.deref()`:

In [ ]:
referenced_var = x.ref().deref()
assert referenced_var is x
referenced_var

## Recursos e leitura adicional

- Veja a seção [Migrando para TF2](https://tensorflow.org/guide/migrate) para ler mais sobre como migrar do TF1.x para TF2.
- Leia o [guia de mapeamento de modelos](./model_mapping.ipynb) para saber mais sobre como mapear seus modelos TF1.x para trabalhar diretamente no TF2. 